# 终极版 使用预处理数据集

In [1]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, hstack

In [11]:

from sklearn.preprocessing import LabelEncoder
import numpy as np
# 读取数据集
# data = pd.read_csv('train.csv')
data = np.load("./train.pkl", allow_pickle=True)
data

vectorizer = CountVectorizer()

vecto = vectorizer.fit_transform(data["text_cleaned"])
csr = csr_matrix(vecto)




In [12]:
import re
def get_urls(text:str):
    pattern = re.compile(r"http\S+")
    urls = pattern.findall(text)
    return urls

def get_mentions(text:str):
    pattern = re.compile(r"@\S+")
    mentions = pattern.findall(text)
    return mentions

data["urls"] = data["text_cleaned"].apply(get_urls)
data["mentions"] = data["text_cleaned"].apply(get_mentions)


In [13]:

le = LabelEncoder()

keywords  = le.fit_transform(data["keyword"].fillna(""))
locations = le.fit_transform(data["location"].fillna(""))
mentions  = le.fit_transform(data["mentions"].apply(lambda x: " ".join(x)).fillna(""))
urls      = le.fit_transform(data["urls"].apply(lambda x: " ".join(x)).fillna(""))





In [14]:



csr = hstack((csr, csr_matrix( keywords.reshape(-1, 1))))
csr = hstack((csr, csr_matrix(locations.reshape(-1, 1))))
csr = hstack((csr, csr_matrix(mentions.reshape(-1, 1))))
csr = hstack((csr, csr_matrix(urls.reshape(-1, 1))))
# print first 5 rows of csr_matrix
print(csr.toarray().shape)

(7613, 16885)


In [15]:

X_train, X_test, y_train, y_test = train_test_split(csr, data['target_relabeled'], test_size=0.2, random_state=42)



In [18]:

# 训练模型
# ramdom forest
from sklearn.ensemble import RandomForestClassifier
# SVM
from sklearn.svm import SVC
# KNN
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
# Naive Bayes
from sklearn.naive_bayes import BernoulliNB, GaussianNB
# naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import ExtraTreeClassifier






clf = ExtraTreeClassifier()
clf.fit(X_train.toarray(), y_train)

# 评估模型性能
y_pred = clf.predict(X_test.toarray())
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7235718975705844
